# Neural Network with 1 hidden layer

### Load Modules

In [1]:
# Set the Random Seed
seed_value= 2022

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import numpy  as np
import pandas  as pd
import datetime
import random
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import tensorflow as tf
from keras.models     import Sequential, load_model
from keras.layers     import Activation, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks  import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1
from tensorflow.keras.losses import MeanSquaredError

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [3]:
def reset_seeds(seed_value=2022):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
reset_seeds()

### Load Dataset - only traditional features

In [4]:
# Load monthly firm characteristics raw data
df = pd.read_parquet('C:/Users/rafae/Documents/HSG/Master Thesis/Data/Final/data07_model_input.parquet')
df = df.sort_values(by=['YM', 'permno'])
df = df.set_index(['year', 'YM', 'permno'])
df

gvkey  reprisk_id         prc         vol         mve_m  \
year YM      permno                                                             
2007 2007-01 10025    11903       37172   45.320000      8086.0  3.700557e+05   
             10026    12825       12684   39.689999      7613.0  7.653725e+05   
             10042    12139        4832    0.720000     26008.0  3.598898e+04   
             10078    12136        1719    6.130000  11333293.0  2.390900e+07   
             10104    12142        4413   16.430000   7234361.0  8.892640e+07   
...                     ...         ...         ...         ...           ...   
2021 2021-12 93304   184167       91339   36.750000    183303.0  1.695898e+06   
             93373   184323       74074    3.020000    528712.0  2.065325e+05   
             93374   184899       64442   74.510002    159495.0  8.587073e+06   
             93423    10567       22547   39.490002    254917.0  3.661156e+06   
             93436   184996       24952  936.719971   6361641.0  1.092218e+09   

                       absacc       acc    aeavol       age       agr  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.698728 -0.745547 -0.646819  0.457506 -0.979644  ...   
             10026   0.577608 -0.635623 -0.393384  0.457506  0.118575  ...   
             10042   0.990840 -0.989822 -0.894148  0.457506 -0.989822  ...   
             10078   0.654962 -0.711959 -0.128753  0.905344 -0.147074  ...   
             10104  -0.014758 -0.107379  0.780153  0.905344  0.770992  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304  -0.855522  0.706269 -0.871045 -0.500896  0.613134  ...   
             93373  -0.213134  0.143881  0.663284 -0.500896 -0.875821  ...   
             93374  -0.514030  0.423284 -0.223881 -0.500896  0.328955  ...   
             93423   0.242985 -0.303881  0.560597  0.143881 -0.584478  ...   
             93436   0.584478 -0.620299 -0.362388 -0.500896  0.856716  ...   

                     sic2_73  sic2_75  sic2_78  sic2_79  sic2_80  sic2_81  \
year YM      permno                                                         
2007 2007-01 10025         0        0        0        0        0        0   
             10026         0        0        0        0        0        0   
             10042         0        0        0        0        0        0   
             10078         0        0        0        0        0        0   
             10104         1        0        0        0        0        0   
...                      ...      ...      ...      ...      ...      ...   
2021 2021-12 93304         0        0        0        0        0        0   
             93373         0        0        0        0        0        0   
             93374         0        0        0        0        0        0   
             93423         0        0        0        1        0        0   
             93436         0        0        0        0        0        0   

                     sic2_82  sic2_83  sic2_87  sic2_99  
year YM      permno                                      
2007 2007-01 10025         0        0        0        0  
             10026         0        0        0        0  
             10042         0        0        0        0  
             10078         0        0        0        0  
             10104         0        0        0        0  
...                      ...      ...      ...      ...  
2021 2021-12 93304         0        0        0        0  
             93373         0        0        0        0  
             93374         0        0        0        0  
             93423         0        0        0        0  
             93436         0        0        0        0  

[338288 rows x 188 columns]

In [5]:
# Select only relevant columns for X and Y (exclude industry & reprisk rating dummy for now)
info_vars = ['YM', 'year', 'permno', 'gvkey', 'reprisk_id', 'prc', 'vol', 'mve_m']
X_vars = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt',
          'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest',
          'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
          'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m',
          'mom6m', 'ms', 'mve', 'mve_ia', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
          'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',
          'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq',
          'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol',
          'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade']
sic2_vars = [col for col in df if col.startswith('sic2')]
reprisk_vars = ['country_sector_average', 'country_sector_average_01', 'current_rri', 'current_rri_01',
                'peak_rri', 'peak_rri_01', 'trend_rri', 'trend_rri_01']
reprisk_rating_vars = [col for col in df if col.startswith('reprisk_rating')]
Y_vars = ['ret', 'ret_wins', 'ret_ex']

# X
X = df[X_vars + sic2_vars]

# Y
Y = df[['ret_ex']]

X.shape, Y.shape

((338288, 162), (338288, 1))

### Hyperparameter Optimization: 4-Fold CV (12y/4 = 3y) and 3y Test

In [6]:
# Training (12y - 80%) and Test set (3y - 20%)
X_trai = X.loc['2007':'2018']
Y_trai = Y.loc['2007':'2018']

X_test = X.loc['2019':'2021']
Y_test = Y.loc['2019':'2021']

In [7]:
# 4-Fold cross validation (9y training and 3y validation)
K_FOLDs = 4
YEARS = list(X_trai.index.unique(level='year').astype('str')) 
TOT = len(YEARS)
TRA = int(TOT* (K_FOLDs-1) / K_FOLDs)
OFF = TOT - TRA

for FOLD in range(K_FOLDs):
    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
    TRAI = [x for x in YEARS if x not in VALI]
    print(VALI, TRAI)    

['2007', '2008', '2009'] ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
['2010', '2011', '2012'] ['2007', '2008', '2009', '2013', '2014', '2015', '2016', '2017', '2018']
['2013', '2014', '2015'] ['2007', '2008', '2009', '2010', '2011', '2012', '2016', '2017', '2018']
['2016', '2017', '2018'] ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [8]:
# Model
model_name = 'NN1_trad'

def create_NN1(l_rate=0.01, l1_pen=0.0001):
    model = Sequential()
    model.add(Dense(32,
                    activation='relu',
                    input_dim=X_trai.shape[1],
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(1))
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=l_rate))
    return model

In [9]:
# Hyperparameter optimization
Y_val_preds = Y_trai.copy()
Y_test_preds = Y_test.copy()
results = []

earlyStopping  = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

model_ix = 0
for l1_pen in [0.001, 0.0005, 0.0001, 0.00005, 0.00001]:
        for l_rate in [0.01, 0.005, 0.001]:
            for ensemble in list(range(0,10)):
                print("•", model_ix, ':',  end=' ')
                    
                for FOLD in range(K_FOLDs):
                    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
                    TRAI = [x for x in YEARS if x not in VALI]
                        
                    mcp_save = ModelCheckpoint(
                        '%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD),
                        save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True
                    )

                    reset_seeds(ensemble*10000)
                        
                    NN_model = create_NN1(l_rate, l1_pen)
                        
                    history = NN_model.fit(
                        X_trai.loc[TRAI], Y_trai.loc[TRAI],
                        validation_data=(X_trai.loc[VALI], Y_trai.loc[VALI]),
                        epochs=200, verbose=0, shuffle=True, batch_size=2000,
                        callbacks=[earlyStopping, mcp_save]
                    )
                    tra_loss = np.min(history.history['loss'])
                    val_loss = np.min(history.history['val_loss'])

                    results.append({
                        'model_ix' :model_ix,
                        'l_rate'   :l_rate,
                        'l1_pen'   :l1_pen,
                        'fold'     :FOLD,
                        'tra_loss' :tra_loss,
                        'val_loss' :val_loss
                    })
                    
                    # Calculate validation predictions
                    best_model = create_NN1(l_rate, l1_pen)
                    best_model.load_weights(r'%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD));
                    Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])
                    
                    # Calculate predictions for test data, if FOLD = 0
                    if FOLD==0:
                        Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)

                model_ix += 1

• 0 : Epoch 7: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 1 : Epoch 7: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 7: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 2 : Epoch 7: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 7: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 3 : Epoc

1929/1929 [==============================] - 3s 2ms/step
Epoch 10: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 10: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 10: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 43 : Epoch 9: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 44 : Epoch 9: early stopping
1929/1929 [==============================] - 3s 2ms/step
Epoch 7: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 8: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 45 : Epoch 8: early stopping
1929

1929/1929 [==============================] - 3s 2ms/step
Epoch 17: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 64 : Epoch 9: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 10: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 10: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 10: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 65 : Epoch 15: early stopping
1929/1929 [==============================] - 3s 2ms/step
Epoch 10: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 11: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 66 : Epoch 11: early stopp

C:\Users\rafae\AppData\Local\Temp\ipykernel_1372\2749122029.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


...

Epoch 59: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 63: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 64: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 149 : Epoch 50: early stopping
   1/1929 [..............................] - ETA: 1:00

C:\Users\rafae\AppData\Local\Temp\ipykernel_1372\2749122029.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


C:\Users\rafae\AppData\Local\Temp\ipykernel_1372\2749122029.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)


Epoch 55: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 47: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 48: early stopping
2066/2066 [==============================] - 4s 2ms/step


In [10]:
# Save Y_val_preds
Y_val_preds.to_csv(r'%s/%s_val_preds.csv'%(model_name, model_name))
Y_val_preds

ret_ex  NN1_trad_0  NN1_trad_1  NN1_trad_2  NN1_trad_3  \
year YM      permno                                                             
2007 2007-01 10025  -0.031894    0.008587    0.009349    0.010351    0.009566   
             10026  -0.042317    0.008587    0.009349    0.010351    0.009566   
             10042  -0.125751    0.008587    0.009349    0.010351    0.009566   
             10078  -0.080607    0.008587    0.009349    0.010191    0.009566   
             10104  -0.046341    0.008587    0.009349    0.010351    0.009566   
...                       ...         ...         ...         ...         ...   
2018 2018-12 93420   0.086508    0.006352    0.006647    0.006925    0.003772   
             93422   0.466817    0.006352    0.006330    0.006925    0.003772   
             93423   0.105036    0.006352    0.006302    0.006925    0.003772   
             93429  -0.048712    0.006352    0.006302    0.006925    0.003772   
             93436  -0.079564    0.006352    0.006302    0.006925    0.003772   

                     NN1_trad_4  NN1_trad_5  NN1_trad_6  NN1_trad_7  \
year YM      permno                                                   
2007 2007-01 10025     0.008143    0.010055    0.010865    0.008627   
             10026     0.008143    0.010055    0.010865    0.008627   
             10042     0.008143    0.010055    0.010865    0.008627   
             10078     0.008143    0.010055    0.010865    0.008627   
             10104     0.008143    0.010055    0.010865    0.008627   
...                         ...         ...         ...         ...   
2018 2018-12 93420     0.003047    0.006652    0.006612    0.006509   
             93422     0.003047    0.006469    0.006612    0.006509   
             93423     0.003047    0.006652    0.006612    0.006509   
             93429     0.003047    0.006652    0.006612    0.006509   
             93436     0.003047    0.006652    0.006612    0.006509   

                     NN1_trad_8  ...  NN1_trad_140  NN1_trad_141  \
year YM      permno              ...                               
2007 2007-01 10025     0.009070  ...      0.021886      0.016970   
             10026     0.009070  ...      0.015572      0.004808   
             10042     0.009070  ...     -0.015591     -0.030024   
             10078     0.009070  ...      0.010335      0.017075   
             10104     0.009070  ...      0.019800      0.019519   
...                         ...  ...           ...           ...   
2018 2018-12 93420     0.005346  ...     -0.023899     -0.028243   
             93422     0.005346  ...     -0.045455     -0.004553   
             93423     0.005346  ...      0.006469      0.002856   
             93429     0.005346  ...      0.006436     -0.002576   
             93436     0.005346  ...      0.019474      0.018258   

                     NN1_trad_142  NN1_trad_143  NN1_trad_144  NN1_trad_145  \
year YM      permno                                                           
2007 2007-01 10025       0.027732      0.025091      0.010518      0.026337   
             10026      -0.000869      0.015770      0.015453      0.010383   
             10042      -0.024133     -0.023104     -0.009637     -0.055110   
             10078       0.009885      0.013778      0.021121      0.006369   
             10104       0.015788      0.017979      0.013834      0.014944   
...                           ...           ...           ...           ...   
2018 2018-12 93420       0.020490     -0.020504      0.007148     -0.028992   
             93422      -0.037616     -0.027460     -0.003842     -0.012458   
             93423       0.008046      0.001537      0.001765      0.001252   
             93429      -0.009258      0.009005      0.003651      0.018193   
             93436      -0.002573      0.028122      0.011312      0.011405   

                     NN1_trad_146  NN1_trad_147  NN1_trad_148  NN1_trad_149  
year YM      permno                                   

In [11]:
# Save Y_test_preds
Y_test_preds.to_csv(r'%s/%s_test_preds.csv'%(model_name, model_name))
Y_test_preds

ret_ex  NN1_trad_0  NN1_trad_1  NN1_trad_2  NN1_trad_3  \
year YM      permno                                                             
2019 2019-01 10026   0.004225    0.008587    0.009349    0.010351    0.009566   
             10104   0.036026    0.008587    0.009349    0.010351    0.009566   
             10107   0.075381    0.008587    0.009349    0.010351    0.009566   
             10138   0.072777    0.008587    0.009349    0.010351    0.009566   
             10145   0.076596    0.008587    0.009349    0.010351    0.009566   
...                       ...         ...         ...         ...         ...   
2021 2021-12 93304  -0.096386    0.008587    0.009349    0.010351    0.009566   
             93373  -0.019481    0.008587    0.009349    0.010351    0.009566   
             93374  -0.047552    0.008587    0.009349    0.010351    0.009566   
             93423  -0.072569    0.008587    0.009349    0.010351    0.009566   
             93436  -0.113609    0.008587    0.009349    0.010351    0.009566   

                     NN1_trad_4  NN1_trad_5  NN1_trad_6  NN1_trad_7  \
year YM      permno                                                   
2019 2019-01 10026     0.008143    0.010055    0.010865    0.008627   
             10104     0.008143    0.010055    0.010865    0.008627   
             10107     0.008143    0.010055    0.010865    0.008627   
             10138     0.008143    0.010055    0.010865    0.008627   
             10145     0.008143    0.010055    0.010865    0.008627   
...                         ...         ...         ...         ...   
2021 2021-12 93304     0.008143    0.010055    0.010865    0.008627   
             93373     0.008143    0.010055    0.010865    0.008627   
             93374     0.008143    0.010055    0.010865    0.008627   
             93423     0.008143    0.010055    0.010865    0.008627   
             93436     0.008143    0.010055    0.010865    0.008627   

                     NN1_trad_8  ...  NN1_trad_140  NN1_trad_141  \
year YM      permno              ...                               
2019 2019-01 10026      0.00907  ...      0.015356      0.007014   
             10104      0.00907  ...      0.020844      0.010480   
             10107      0.00907  ...      0.013457      0.012707   
             10138      0.00907  ...      0.027886     -0.002468   
             10145      0.00907  ...      0.025624     -0.000459   
...                         ...  ...           ...           ...   
2021 2021-12 93304      0.00907  ...     -0.004481      0.013276   
             93373      0.00907  ...     -0.007696     -0.048221   
             93374      0.00907  ...      0.002938      0.012269   
             93423      0.00907  ...     -0.017845      0.019938   
             93436      0.00907  ...      0.020240     -0.008122   

                     NN1_trad_142  NN1_trad_143  NN1_trad_144  NN1_trad_145  \
year YM      permno                                                           
2019 2019-01 10026       0.001944      0.016630      0.004285      0.008731   
             10104       0.011895      0.011385      0.004571      0.009143   
             10107       0.007951      0.010619      0.013397      0.016639   
             10138       0.008548      0.012469      0.010582      0.010583   
             10145       0.004180      0.005044      0.010669      0.011504   
...                           ...           ...           ...           ...   
2021 2021-12 93304       0.024475      0.021797      0.004262      0.007859   
             93373      -0.015251     -0.019141     -0.026365     -0.034568   
             93374       0.008290      0.010100      0.014642      0.008294   
             93423       0.004462      0.003850      0.008786      0.004799   
             93436      -0.004667      0.011542      0.015574     -0.009626   

                     NN1_trad_146  NN1_trad_147  NN1_trad_148  NN1_trad_149  
year YM      permno                                   

In [12]:
# Result overview
table = pd.DataFrame(results)
table = table.groupby(['model_ix', 'l_rate', 'l1_pen']).mean().sort_values('val_loss')
table.to_csv(r'%s/%s_results.csv'%(model_name, model_name))

np.sqrt(table[['tra_loss', 'val_loss']].head(20)) * 100

,,,tra_loss,val_loss
model_ix,l_rate,l1_pen,,
84,0.001,0.00010,12.982838,12.977049
85,0.001,0.00010,12.982037,12.978578
89,0.001,0.00010,12.988923,12.980617
86,0.001,0.00010,12.986199,12.980716
80,0.001,0.00010,12.982944,12.982155
83,0.001,0.00010,12.985114,12.982438
82,0.001,0.00010,12.985813,12.983226
87,0.001,0.00010,12.985182,12.987430
115,0.001,0.00005,12.972748,12.994716


## Model Selection: Maximize SR on LS-Portfolio on validation data

In [13]:
# Model
model_name = 'NN1_trad'

# Get predictions
val = pd.read_csv(r'%s/%s_val_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])
test = pd.read_csv(r'%s/%s_test_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])

# Get overview table
table = pd.read_csv(r'%s/%s_results.csv'%(model_name, model_name)).sort_values(['model_ix'])

# Extract hyperparameters
list_l_rate = list(pd.unique(table['l_rate']))
list_l1_pen = list(pd.unique(table['l1_pen']))

In [14]:
# Loop over hyperparameter combinations to get ensemble predictions for each hyperparameter combination
val_results = []
hyper_comb = 0
for l_rate in list_l_rate:
    for l1_pen in list_l1_pen:
        # Get list of relevant model_ix
        list_model_ix = table[(table['l_rate'] == l_rate) & (table['l1_pen'] == l1_pen)]
        list_model_ix = list(list_model_ix['model_ix'])
        # Get list of column names
        list_col_names = []
        for ele in list_model_ix:
            list_col_names.append('%s_%d'%(model_name, ele))
        # Select relevant returns and average predictions
        val_ret = val[list_col_names]
        val_ret = val_ret.mean(axis=1)
        # Append ensemble return prediction to true returns
        comb = val[['ret_ex']].copy()
        comb['ret_pred'] = val_ret
        comb = comb.reset_index()
        # Sort the data by predicted returns and divide the data into quintiles
        comb['quintile'] = comb.groupby(['YM'])['ret_pred'].transform(lambda x: pd.qcut(x.rank(method='first'), 5, labels=np.arange(1,6)))
        # Calculate the mean return for each YM
        comb_mean = comb.groupby(['YM', 'quintile']).agg(ret_ex = ('ret_ex', 'mean'))
        # Add LS-Strategy
        comb_mean = comb_mean[['ret_ex']].unstack().add_prefix('Q')
        comb_mean.columns = comb_mean.columns.droplevel(0)
        comb_mean.columns.name = None
        comb_mean['LS'] = comb_mean['Q5'] - comb_mean['Q1']
        # Calculate the average return, standard deviation and Sharpe Ratio (annualized) per Quintile
        summary = pd.DataFrame()
        summary['mean'] = comb_mean.mean()
        summary['std'] = comb_mean.std()
        summary['SR'] = summary['mean'] / summary['std'] * np.sqrt(12)
        # Append results
        val_results.append({
            'hyper_comb':hyper_comb,
            'l_rate'    :l_rate,
            'l1_pen'    :l1_pen,
            'SR_Q1'     :summary.loc['Q1','SR'],
            'SR_Q5'     :summary.loc['Q5','SR'],
            'SR_LS'     :summary.loc['LS','SR'],
            'Mean_Q1'   :summary.loc['Q1','mean'],
            'Mean_Q5'   :summary.loc['Q5','mean'],
            'Mean_LS'   :summary.loc['LS','mean'],
        })
        hyper_comb += 1
        
# Save results
val_results = pd.DataFrame(val_results)
val_results.to_csv(r'results/{}_val.csv'.format(model_name))

In [15]:
# Select best hyperparemeters (max. SR_LS)
val_results = val = pd.read_csv(r'results/{}_val.csv'.format(model_name))
val_results = val_results.set_index(['l_rate', 'l1_pen']).sort_values(['SR_LS'], ascending=False)
l_rate_opt, l1_pen_opt = tuple([x for x in val_results.iloc[0].name[0:]])
print('Optimal l_rate:', l_rate_opt)
print('Optimal l1_pen:', l1_pen_opt)
val_results

Optimal l_rate: 0.001
Optimal l1_pen: 1e-05


Unnamed: 0  hyper_comb     SR_Q1     SR_Q5     SR_LS  \
l_rate l1_pen                                                          
0.001  0.00001          14          14  0.065159  0.493423  1.099535   
       0.00005          13          13  0.032590  0.528825  0.812886   
0.005  0.00001           9           9  0.068998  0.474388  0.792958   
0.010  0.00001           4           4  0.051003  0.460202  0.790610   
0.001  0.00010          12          12  0.047260  0.542114  0.648000   
       0.00050          11          11 -0.018961  0.703116  0.595547   
       0.00100          10          10 -0.001838  0.675991  0.585600   
0.010  0.00005           3           3  0.018425  0.537598  0.585233   
0.005  0.00005           8           8  0.021574  0.519464  0.572196   
       0.00010           7           7  0.037053  0.605588  0.555355   
0.010  0.00010           2           2  0.031782  0.557422  0.552937   
0.005  0.00050           6           6  0.060433  0.616570  0.523245   
       0.00100           5           5  0.179797  0.587657  0.284935   
0.010  0.00050           1           1  0.264310  0.295341  0.014528   
       0.00100           0           0  0.384905  0.262857 -0.237349   

                 Mean_Q1   Mean_Q5   Mean_LS  
l_rate l1_pen                                 
0.001  0.00001  0.001358  0.008859  0.007501  
       0.00005  0.000756  0.008964  0.008207  
0.005  0.00001  0.001512  0.008028  0.006516  
0.010  0.00001  0.001131  0.007891  0.006760  
0.001  0.00010  0.001132  0.008755  0.007622  
       0.00050 -0.000468  0.008936  0.009404  
       0.00100 -0.000045  0.008684  0.008729  
0.010  0.00005  0.000445  0.007816  0.007371  
0.005  0.00005  0.000521  0.007735  0.007215  
       0.00010  0.000921  0.008845  0.007924  
0.010  0.00010  0.000776  0.008126  0.007350  
0.005  0.00050  0.001427  0.008534  0.007107  
       0.00100  0.004085  0.007628  0.003544  
0.010  0.00050  0.005094  0.005181  0.000087  
       0.00100  0.006529  0.005134 -0.001396

## Out-of-Sample: Get Predictions of model with best hyperparameter combination

In [16]:
# Get list of model_ix's with best hyperparameter combinations and make ensemble predictions
list_model_ix = table[(table['l_rate'] == l_rate_opt) & (table['l1_pen'] == l1_pen_opt)]
list_model_ix = list(list_model_ix['model_ix'])

# Get list of column names
list_col_names = []
for ele in list_model_ix:
    list_col_names.append('%s_%d'%(model_name, ele))
    
# Select relevant returns and average predictions
test_ret = test[list_col_names].copy()
test_ret['y_pred'] = test_ret.mean(axis=1)

# Save predictions
test_ret = test_ret.reset_index()
test_ret = test_ret[['YM', 'permno', 'y_pred']]
test_ret.to_csv(r'results/{}_predictions.csv'.format(model_name), index=False)